In [2]:
import kivy
from kivy.app import App 
from kivy.uix.label import Label
from kivy.uix.gridlayout import GridLayout
from kivy.uix.textinput import TextInput
from kivy.uix.button import Button
from kivy.uix.togglebutton import ToggleButton
from kivy.uix.widget import Widget
from kivy.properties import ObjectProperty
from kivy.uix.floatlayout import FloatLayout
from kivy.graphics import Rectangle
from kivy.graphics import Color
from kivy.graphics import Line
from kivy.lang import Builder
from kivy.uix.screenmanager import ScreenManager, Screen
from kivy.core.window import Window
from kivy.properties import StringProperty
from kivy.properties import NumericProperty
from kivy.properties import ListProperty

Window.size = (800, 800)

calc_list = [""] * 9
print(calc_list)

#Note to self: If I throw data pull here, would it become a global variable 
#              and work when I call it in each class

class ConnectionWindow(Screen):

    #Save Connection Type Chosen
    connection = ObjectProperty(None)
    def connection_fixed(self):
        connection = "fixed"
        print(connection)
        ConnectionWindow.connection = connection
        calc_list [0] = connection
    def connection_pin (self):
        connection = "pin"
        print(connection)
        ConnectionWindow.connection = connection
        calc_list [0] = connection


class VariablesWindow (Screen):
    
    #Save Load Type Chosen
    load = ObjectProperty(None)
    def load_distributed(self):
        load = "distributed"
        VariablesWindow.load = load
        calc_list [1] = load
    def load_point(self):
        load = "point"
        VariablesWindow.load = load
        calc_list [1] = load
    def load_moment(self):
        load = "moment"
        VariablesWindow.load = load
        calc_list [1] = load   

    #Save Material Type
    material = ObjectProperty(None)
    def mat_wood(self):
        material = "wood"
        VariablesWindow.material = material
        calc_list[8] = material
    
    def mat_steel(self):
        material = "steel"
        VariablesWindow.material = material
        calc_list[8] = material

    def mat_aluminum(self):
        material = "aluminum"
        VariablesWindow.material = material
        calc_list[8] = material


    #Save Input Variables
    def save_variables(self):
        calc_list [2] = load_value = self.ids.load_value.text
        calc_list [3] = a_value = self.ids.a_value.text
        calc_list [4] = r_value = self.ids.r_value.text
        calc_list [5] = b_value = self.ids.b_value.text
        calc_list [6] = h_value = self.ids.h_value.text
        calc_list [7] = l_value = self.ids.l_value.text
        print(calc_list)

    #Pass Variables through Calculator, Output Result
    def calculate_variables(self):        
        #Transfer Variables from Calc_list into instance of variable
        connection_final = calc_list[0]
        load_type_final = calc_list[1]
        load_final = (calc_list[2])
        a_final = (calc_list[3])
        #r_final = (calc_list[4])
        b_final = (calc_list[5])        
        h_final = (calc_list[6])
        l_final = (calc_list[7])
        material_final = calc_list[8]
        error_message = str("")

        #Troubleshoot Missing Variables Before They Are Passed
        if connection_final == "":
            error_message = "Connection not chosen"
            print(error_message)
            return
        
        if load_type_final == "":
            error_message = "Load type not chosen"
            print(error_message)
            return

        if load_final == "":
            error_message = "Load magnitude not entered"
            print(error_message)
            return

        if a_final == "":
            error_message = "Distance A not entered"
            print(error_message)
            return

        if l_final == "":
            error_message = "Length of beam not entered"
            print(error_message)
            return

        if material_final == "":
            error_message = "Material type not chosen"
            print(error_message)
            return          

        #Check Beam Dimensions 
        if h_final == "" or b_final == "":
            error_message = "Beam Dimensions not Input"
            print(error_message)
            return
        
        #Convert Numbers to Floats if Troubleshoot Test Passes
        load_final = float(calc_list[2])
        a_final = float(calc_list[3])
        #r_final = float(calc_list[4])
        b_final = float(calc_list[5])
        h_final = float(calc_list[6])
        l_final = float(calc_list[7])

        #Calculate Beam Properties
        area = h_final * b_final
        moi = (1/12)*b_final*(h_final)**4
        print(area, moi)

        print("continue?")
  

class OutputWindow (Screen):
    pass

class WindowManager(ScreenManager):
    load = StringProperty("")
    connection = StringProperty("")
    mat = StringProperty("")
    pass

kv = Builder.load_file("my.kv")

class MyApp (App):
    def build (self):
        return kv

#Run Program 
if __name__ == "__main__":
    MyApp().run()



['', '', '', '', '', '', '', '', '']
fixed
['fixed', '', '', '', '', '', '', '', '']
Load type not chosen
['fixed', 'distributed', '', '', '', '', '', '', '']
Load magnitude not entered
['fixed', 'distributed', '1', '', '', '', '', '', '']
Distance A not entered
['fixed', 'distributed', '1', '1', '', '', '', '', '']
Length of beam not entered
['fixed', 'distributed', '1', '1', '', '', '', '1', '']
Material type not chosen
['fixed', 'distributed', '1', '1', '', '', '', '1', 'steel']
Beam Dimensions not Input
['fixed', 'distributed', '1', '1', '1', '', '', '1', 'steel']
Beam Dimensions not Input
['fixed', 'distributed', '1', '1', '1', '3', '', '1', 'steel']
Beam Dimensions not Input
['fixed', 'distributed', '1', '1', '1', '3', '4', '1', 'steel']
12.0 64.0
continue?
['fixed', 'distributed', '1', '1', '1', '', '4', '1', 'steel']
['fixed', 'distributed', '1', '1', '1', '', '4', '1', 'steel']
Beam Dimensions not Input
['fixed', 'distributed', '1', '1', '1', '2', '', '1', 'steel']
Beam Dimens

KeyboardInterrupt: 